In [1]:
import numpy as np
import time
import random

In [108]:
def mvrsample(A, **kwargs):
    
    # Check the input adjacent matrix
    if A.size == 0 or A.shape[0] != A.shape[1]:
        print('Error: input ''A'' must be a non-empty square matrix.')
        return
    
    if np.any(A<0) or (np.setdiff1d(A, np.floor(A)).size > 0):
        print('Error: input ''A'' elements must be natural numbers.')
        return
    
    # Initialize
    n = A.shape[0] # node
    m = A.sum() # edge
    
    s_rate = n # sampling interval
    n_samp = n
    Tb     = n*n # warm start
    quiet  = False
        
    if not quiet:
        print('Minimum violation ranking sampler')
        print('   nodes, n = %i' % n)
        print('   edges, m = %i' % m)
        print('   steps between samples   = %i' % s_rate)
        print('   target sample count     = %i' % n_samp)
    
    # initialize the ranking out-degree, in decreasing order
    kout = np.sum(A, axis=1)  # sum row, get the out-degrees
    I = np.argsort(kout)[::-1]  # sort them（descending order), return index
    h = np.zeros(n, dtype=int)  # the ranking vector
    for i in range(n):
        h[I[i]] = i  # update the ranking vector

    # initialize the MVR score（defined as the number of non-violate edge)              
    score = m
    for i in range(n):
        for j in range(n):
            if A[i,j] > 0:
                if h[i] > h[j]:
                    score -= A[i, j]        
    print("score： "+ str(score))
    
    if not quiet:
        print('[t=%4.0f] violations = %i (%3.1f%%)\twarm-start_rest: %i\n' % (
            1, m - score, 100 * (1 - score / m), Tb))
    maxs = score

    # zero-temperature MCMC (Markov chain Monte Carlo method) sampler
    rs = np.zeros((n, n_samp))  
    k = 0 # Sampling counter
    T = Tb + n_samp * s_rate
    t = 0  

    while True:
        t += 1
        #  check stopping criteria
        if t > T: 
            break
        
        # choose 2 positions to swap
        h2 = h.copy()
        p1 = np.random.randint(n)
        p2 = p1
        while p1 == p2:
            p1 = np.random.randint(n)
            p2 = np.random.randint(n)
            
        temp1, temp2 = h2[p1], h2[p2]
        h2[p1], h2[p2] = temp2, temp1
        
        # calculate new score
        new = m
        for i in range(n):
            for j in range(n):
                if A[i,j] > 0:
                    if h2[i] > h2[j]:
                        new -= A[i, j]
                        
        # Determines whether to replace the original ranking
        if new > maxs:
            h = h2.copy() 
            maxs = new
            if not quiet and t > Tb:
                print(f'[t={t}] violations = {m-maxs} ({100*(1-score/m):3.1f}%) found a better MVR; restarting sampling')
            # The sample is cleared and resampled for better results
            if t > Tb:
                k, t = 0, Tb+1
            
            
        # Sampling
        if t > Tb and t % s_rate == 0:
            rs[:, k] = h.copy() 
            k += 1 

        # update the best result to user
        if t % 1000 == 0:
            if not quiet:
                if t <= Tb:
                    print('[t=%i] violations = %i (%3.1f%%)\twarm-start_rest: %i' % 
                          (t, m-maxs, 100*(1-maxs/m), Tb-t))
                else:
                    print('[t=%i] violations = %i (%3.1f%%)\tsamples: %i' % 
                          (t, m-maxs, 100*(1-maxs/m), k))

        # Recheck stopping criteria
        if t > T:
            break

    # Store the results
    pi = np.zeros((n, 1)) # mean of ranks across MVR samples
    pi[:, 0] = np.mean(rs, axis=1)
    
    # assign the rank according to the mean
    ranks = np.column_stack(((np.arange(n)), np.mean(pi, axis=1)))
    ranks = ranks[ranks[:,1].argsort()] # sort the second colume 

    I = np.argsort(np.mean(pi, axis=1))
    h = np.zeros(n, dtype=int)
    
    for i in range(n):
        h[I[i]] = i
        
    violate = 0
    for i in range(n):
        for j in range(n):
            if A[i,j] > 0:
                if h[i] > h[j]:
                    violate += 1      
    print('violate number： '+ str(violate))
    # Fraction of edges that violate the ranking
    rho = violate/m
    print('violate rate: '+ str(rho))

    return ranks

In [ ]:
n = 205
A = np.zeros((n, n))

with open('../ComputerScience_edgelist.csv', 'r') as fileID:
    while True:
        line = fileID.readline()
        if not line:
            break
        data = line.strip().split(',')
        i = int(data[0])
        j = int(data[1])
        # index begin with 0
        A[i-1, j-1] += 1
fileID.close()

ranks = mvrsample(A)

with open('output_rank.txt', 'w') as f:
    f.write('my rank  my score    supplementary material\'s rank \n')
    for i in range(len(ranks)):
        f.write(str(i)+ '\t' + str(ranks[i][1])[0:7] + '\t' + str(ranks[i][0])  + '\n')

In [110]:
m=4388

cnt = 0
with open('../ComputerScience_edgelist.csv', 'r') as fileID:
    while True:
        line = fileID.readline()
        if not line:
            break
        data = line.strip().split(',')
        i = int(data[0])
        j = int(data[1])
        if i > j:
            cnt += 1
fileID.close()

print('violate number： '+ str(cnt))
rho = cnt/m
print('violate rate: '+ str(rho))

violate number： 484
violate rate: 0.11030082041932543
